#Libraries And GDrive Connection

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/YL TEZ/py/swan_models/new_models/Spectrums

/gdrive/MyDrive/YL TEZ/py/swan_models/new_models/Spectrums


In [2]:
import pandas as pd
import os

In [ ]:
cases = os.listdir()
case = cases[85]
case

'09260100_Fit_Data.xlsx'

#Define SET LEVEL

In [ ]:
date = case[0:8] #"09160100" #"09260100" #"09271300"
date

'09260100'

In [ ]:
year = "1994"
month = date[0:2]
day = date[2:4]
hour = date[4:8]

In [ ]:
date_time_tide = year + "-" + month + "-" + day + "T" + hour[0:2] + ":" + hour[2:] + ":00Z"
date_time_tide

'1994-09-26T01:00:00Z'

In [ ]:
if not int(hour[0:2])-1 < 10:
  date_time_tide1 = year + "-" + month + "-" + day + "T" + str(int(hour[0:2])-1) + ":" + "59" + ":59Z"
else:
  date_time_tide1 = year + "-" + month + "-" + day + "T0" + str(int(hour[0:2])-1) + ":" + "59" + ":59Z"
date_time_tide1

'1994-09-26T00:59:59Z'

In [3]:
os.chdir("/gdrive/MyDrive/YL TEZ/py/swan_models/new_models/Profiles_Excel")

In [ ]:
tide = pd.read_csv('noaa_tide.csv')
try:
  set_level = float(tide.loc[tide["date_time"].str.contains(date_time_tide)].iloc[0, 1])
except:
  set_level = float(tide.loc[tide["date_time"].str.contains(date_time_tide1)].iloc[0, 1])
print(set_level)

-0.091


#Copy model dir

In [ ]:
import os
import shutil

new_dir = r"model_" + date
source_folder = r"/gdrive/MyDrive/YL TEZ/py/swan_models/ppp87/"
destination_folder1 = r"/gdrive/MyDrive/YL TEZ/py/swan_models/new_models/"

create_path = os.path.join(destination_folder1, new_dir)
os.mkdir(create_path)
destination_folder = destination_folder1 + new_dir + "/"


print(destination_folder)
# fetch all files
for file_name in os.listdir(source_folder):
    #print(file_name)
    # construct full file path
    source = source_folder + file_name
    destination = destination_folder + file_name
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)
        print('copied', file_name)
    else: print(source)

FileExistsError: ignored

#Get min_x

In [ ]:
profile = profile.loc[(profile['Elevation'] <= 0)]

# Bottom Profile Interpolation

In [512]:
os.chdir("/gdrive/MyDrive/YL TEZ/py/swan_models/new_models/Profiles_Excel")

In [513]:
pro_files = os.listdir()
len(pro_files)

40

In [514]:
date_new = pro_files[nb][4:8]
year = '1994'

In [515]:
only_date = year + date_new
only_date

'19940818'

In [516]:
profile = pd.read_excel(only_date + ".xlsx", index_col=0)

In [517]:
interp_x = float(profile.iloc[-1, :]["x"])
interp_y = float(profile.iloc[-1, :]["Elevation"])

In [518]:
from scipy import interpolate
import numpy as np
x = np.array([interp_x, 873.839])
y = np.array([interp_y, -7.864])
f = interpolate.interp1d(x, y, fill_value='extrapolate')
xnew = np.arange(interp_x+1, 886, 1)
ynew = f(xnew)

In [519]:
xnew2=[]
for xj in range(len(xnew)):
  xnew2.append(int(xnew[xj]))

In [520]:
concat_df = pd.DataFrame({"x": xnew2, "Elevation": ynew})

In [521]:
profile = pd.concat([profile, concat_df], ignore_index=True)

In [522]:
os.chdir("/gdrive/MyDrive/YL TEZ/py/swan_models/new_models/Profiles_Excel/uzun")

In [523]:
profile.to_excel(only_date + '.xlsx')
%ls

19940803.xlsx  19940909.xlsx  19941003.xlsx  19941011.xlsx  19941019.xlsx
19940810.xlsx  19940914.xlsx  19941004.xlsx  19941012.xlsx  19941020.xlsx
19940812.xlsx  19940916.xlsx  19941005.xlsx  19941013.xlsx  19941021.xlsx
19940816.xlsx  19940920.xlsx  19941006.xlsx  19941014.xlsx  19941026.xlsx
19940818.xlsx  19940921.xlsx  19941007.xlsx  19941015.xlsx  19941109.xlsx
19940820.xlsx  19940924.xlsx  19941008.xlsx  19941016.xlsx  19941121.xlsx
19940825.xlsx  19940926.xlsx  19941009.xlsx  19941017.xlsx
19940907.xlsx  19940930.xlsx  19941010.xlsx  19941018.xlsx


In [511]:
nb +=1
nb

4

# nokta


In [ ]:
profile11 = profile.sort_values(by='Elevation', ascending=True)['Elevation'].multiply(-1)

In [ ]:
min_x = str(int(profile.iloc[-1,0:1])*-1)
min_x

'-885'

Copied Files

#profile11.bot File

In [ ]:
os.chdir(destination_folder)

In [ ]:
num_lines = profile11.to_string(index=False).replace(" ", "").count("\n")+1
num_lines

760

In [ ]:
metin = str(profile11.to_string(index=False).replace(" ", ""))

In [ ]:
metin = metin.replace(".", ",")

In [ ]:
metin = metin.encode("utf-8")

In [ ]:
my_file = open("profile11.bot", "wb")
my_file.write(metin)

6839

In [ ]:
my_file.close()

In [ ]:
print(num_lines)

760


In [ ]:
mesh_x = num_lines+1

#Creating SWN file

In [ ]:
my_file = open("nodissip2.SWN", "w")
line1 = """PROJECT 'project_003''A577' '1'  
MODE STATIONARY ONED        
SET DEPMIN=0.01 MAXMES=999 MAXERR=3 PWTAIL=4  
"""
line2 = "SET LEVEL " + str(set_level)
line3 = """
SET CARTESIAN             
SETUP 0               
"""
line4 = "CGRID REGULAR "+ min_x +" 0 0 "+ str(mesh_x) +" 0. "+ str(mesh_x-1) +" 0 &                         "
line5 = """
CIRCLE 180 0.04 1.4961 38       
!BOUND SHAPE JONSWAP 3.3 PEAK DSPR POWER               
""" 
line6 = "INPGRID BOTTOM REGULAR " + min_x + " 0 0 "+ str(mesh_x-1) +" 0 1 1              "
line7 ="""
READ BOTTOM 1. 'profile11.bot' 1 0 FREE                        
!NPGRID CURRENT REGULAR 0 0 0 100 0 10 1             
!EAD CURRENT                                                          
BOUN SIDE W CONSTANT FILE 'sinir.SP1' 1    
                          
WIND 0 00        
GEN3 KOMEN       
!BREAK 1. 0.73            
!OFF WCAP
!OFF BREAKING
!FRIC JONSWAP           
TRIAD             
NUM ACCUR 0.02 0.02 0.02 98 15          
OUTPUT OPTIONS '%' TABLE 16 BLOCK 9 1000 SPEC 8       
"""
line8 = "CURVE 'curve' "+ min_x + " 0 " + str(mesh_x)+ " " + str(mesh_x) + " 0                    "
line9 = """
Table 'curve' HEADER 'project_003.tab' XP YP HSIGN RTP TM01 TMM10 DIR &  
DSPR DEPTH SETUP                 
POIN 'PALL' FILE 'points.txt'
SPEC 'PALL' SPEC1D ABS 'PALL.sp1'
SPEC 'PALL' SPEC2D ABS 'PALL.sp2'


TEST 0 0         
COMPUTE
STOP
"""

wir = line1+line2+line3+line4+line5+line6+line7+line8+line9
print(wir)
my_file.write(wir)

PROJECT 'project_003''A577' '1'  
MODE STATIONARY ONED        
SET DEPMIN=0.01 MAXMES=999 MAXERR=3 PWTAIL=4  
SET LEVEL -0.091
SET CARTESIAN             
SETUP 0               
CGRID REGULAR -885 0 0 761 0. 760 0 &                         
CIRCLE 180 0.04 1.4961 38       
!BOUND SHAPE JONSWAP 3.3 PEAK DSPR POWER               
INPGRID BOTTOM REGULAR -885 0 0 760 0 1 1              
READ BOTTOM 1. 'profile11.bot' 1 0 FREE                        
!NPGRID CURRENT REGULAR 0 0 0 100 0 10 1             
!EAD CURRENT                                                          
BOUN SIDE W CONSTANT FILE 'sinir.SP1' 1    
                          
WIND 0 00        
GEN3 KOMEN       
!BREAK 1. 0.73            
!OFF WCAP
!OFF BREAKING
!FRIC JONSWAP           
TRIAD             
NUM ACCUR 0.02 0.02 0.02 98 15          
OUTPUT OPTIONS '%' TABLE 16 BLOCK 9 1000 SPEC 8       
CURVE 'curve' -885 0 761 761 0                    
Table 'curve' HEADER 'project_003.tab' XP YP HSIGN RTP TM01 TMM10 DIR &  
DSP

1162

In [ ]:
my_file.close()

In [ ]:
os.chdir("/gdrive/MyDrive/YL TEZ/py/swan_models/new_models/Spectrums")

In [ ]:
date

'09260100'

In [ ]:
spec_file = pd.ExcelFile(date + "_Fit_Data.xlsx")#, index_col=0) #, sheet_name=None)

In [ ]:
spec_file.sheet_names

['09260100.ppp02',
 '09260100.ppp03',
 '09260100.ppp04',
 '09260100.ppp14',
 '09260100.ppp13',
 '09260100.ppp05',
 '09260100.ppp12',
 '09260100.ppp23',
 '09260100.ppp16',
 '09260100.ppp15',
 '09260100.ppp19',
 '09260100.ppp17',
 '09260100.ppp18',
 '09260100.ppp45',
 '09260100.ppp87']

## Find boundary sensor

In [ ]:
p87 = date + ".ppp87"
p19 = date + ".ppp19"
p18 = date + ".ppp18"
p45 = date + ".ppp45"
p17 = date + ".ppp17"
p15 = date + ".ppp15"
p16 = date + ".ppp16"
p14 = date + ".ppp14"
p13 = date + ".ppp13"
p12 = date + ".ppp12"
p26 = date + ".ppp26"
p23 = date + ".ppp23"
p05 = date + ".ppp05"
p04 = date + ".ppp04"
p03 = date + ".ppp03"
p02 = date + ".ppp02"
p01 = date + ".ppp01"

In [ ]:
if p87 in spec_file.sheet_names:
  sheetname = p87
  boun_loc = "-884.1100"
elif p19 in spec_file.sheet_names:
  sheetname = p19
  boun_loc = "-480.3400"
elif p18 in spec_file.sheet_names:
  sheetname = p18
  boun_loc = "-398.3900"
elif p45 in spec_file.sheet_names:
  sheetname = p45
  boun_loc = "-370.0800"
elif p17 in spec_file.sheet_names:
  sheetname = p17
  boun_loc = "-320.3700"
elif p15 in spec_file.sheet_names:
  sheetname = p15
  boun_loc = "-264.7000"
elif p16 in spec_file.sheet_names:
  sheetname = p16
  boun_loc = "-252.0300"
elif p14 in spec_file.sheet_names:
  sheetname = p14
  boun_loc = "-240.55000"
elif p13 in spec_file.sheet_names:
  sheetname = p13
  boun_loc = "-220.2300"
elif p12 in spec_file.sheet_names:
  sheetname = p12
  boun_loc = "-205.3400"
elif p26 in spec_file.sheet_names:
  sheetname = p26
  boun_loc = "-190.2000"
elif p23 in spec_file.sheet_names:
  sheetname = p23
  boun_loc = "-190.2000"
elif p05 in spec_file.sheet_names:
  sheetname = p05
  boun_loc = "-169.5000"
elif p04 in spec_file.sheet_names:
  sheetname = p04
  boun_loc = "-160.7700"
elif p03 in spec_file.sheet_names:
  sheetname = p03
  boun_loc = "-145.4200"
elif p02 in spec_file.sheet_names:
  sheetname = p02
  boun_loc = "-135.0100"
elif p01 in spec_file.sheet_names:
  sheetname = p01
  boun_loc = "-124.9400"
print(sheetname)
print(boun_loc)

09260100.ppp87
-884.1100


In [ ]:
spec = pd.read_excel(date + "_Fit_Data.xlsx", sheet_name=sheetname, index_col=0)
spec

,Freq,PSSE,PSSE (Fit)
0,0.0176,6.050000e-03,0.006050
1,0.0225,1.020000e-02,0.010200
2,0.0273,1.020000e-02,0.010200
3,0.0322,1.250000e-02,0.012500
4,0.0371,1.390000e-02,0.013900
...,...,...,...
196,0.9746,4.450000e+20,0.000155
197,0.9795,8.170000e+20,0.000149
198,0.9844,1.570000e+21,0.000143
199,0.9893,3.120000e+21,0.000138


## Write sinir.sp1

In [ ]:
from decimal import Decimal

In [ ]:
'%.2E' % Decimal('40800000000.00000000000000')

'4.08E+10'

In [ ]:
spec['Freq']

0      0.0176
1      0.0225
2      0.0273
3      0.0322
4      0.0371
        ...  
196    0.9746
197    0.9795
198    0.9844
199    0.9893
200    0.9941
Name: Freq, Length: 201, dtype: float64

In [ ]:
spec['Freq'].apply(lambda x: '%.2E' % Decimal (x)).to_string(index=False).replace(" ", "").replace("\n",  " 0.00 22.5\n")

'1.76E-02 0.00 22.5\n2.25E-02 0.00 22.5\n2.73E-02 0.00 22.5\n3.22E-02 0.00 22.5\n3.71E-02 0.00 22.5\n4.20E-02 0.00 22.5\n4.69E-02 0.00 22.5\n5.18E-02 0.00 22.5\n5.66E-02 0.00 22.5\n6.15E-02 0.00 22.5\n6.64E-02 0.00 22.5\n7.13E-02 0.00 22.5\n7.62E-02 0.00 22.5\n8.11E-02 0.00 22.5\n8.59E-02 0.00 22.5\n9.08E-02 0.00 22.5\n9.57E-02 0.00 22.5\n1.01E-01 0.00 22.5\n1.05E-01 0.00 22.5\n1.10E-01 0.00 22.5\n1.15E-01 0.00 22.5\n1.20E-01 0.00 22.5\n1.25E-01 0.00 22.5\n1.30E-01 0.00 22.5\n1.35E-01 0.00 22.5\n1.40E-01 0.00 22.5\n1.45E-01 0.00 22.5\n1.49E-01 0.00 22.5\n1.54E-01 0.00 22.5\n1.59E-01 0.00 22.5\n1.64E-01 0.00 22.5\n1.69E-01 0.00 22.5\n1.74E-01 0.00 22.5\n1.79E-01 0.00 22.5\n1.84E-01 0.00 22.5\n1.89E-01 0.00 22.5\n1.93E-01 0.00 22.5\n1.98E-01 0.00 22.5\n2.03E-01 0.00 22.5\n2.08E-01 0.00 22.5\n2.13E-01 0.00 22.5\n2.18E-01 0.00 22.5\n2.23E-01 0.00 22.5\n2.27E-01 0.00 22.5\n2.32E-01 0.00 22.5\n2.37E-01 0.00 22.5\n2.42E-01 0.00 22.5\n2.47E-01 0.00 22.5\n2.52E-01 0.00 22.5\n2.57E-01 0.00 22.5\

In [ ]:
size = spec['Freq'].size

In [ ]:
spec['Freq'].to_string(index=False).replace(" ", "\t")

'0.0176\n0.0225\n0.0273\n0.0322\n0.0371\n0.0420\n0.0469\n0.0518\n0.0566\n0.0615\n0.0664\n0.0713\n0.0762\n0.0811\n0.0859\n0.0908\n0.0957\n0.1006\n0.1055\n0.1104\n0.1152\n0.1201\n0.1250\n0.1299\n0.1348\n0.1396\n0.1445\n0.1494\n0.1543\n0.1592\n0.1641\n0.1689\n0.1738\n0.1787\n0.1836\n0.1885\n0.1934\n0.1982\n0.2031\n0.2080\n0.2129\n0.2178\n0.2227\n0.2275\n0.2324\n0.2373\n0.2422\n0.2471\n0.2520\n0.2568\n0.2617\n0.2666\n0.2715\n0.2764\n0.2813\n0.2861\n0.2910\n0.2959\n0.3008\n0.3057\n0.3105\n0.3154\n0.3203\n0.3252\n0.3301\n0.3350\n0.3398\n0.3447\n0.3496\n0.3545\n0.3594\n0.3643\n0.3691\n0.3740\n0.3789\n0.3838\n0.3887\n0.3936\n0.3984\n0.4033\n0.4082\n0.4131\n0.4180\n0.4229\n0.4277\n0.4326\n0.4375\n0.4424\n0.4473\n0.4521\n0.4570\n0.4619\n0.4668\n0.4717\n0.4766\n0.4814\n0.4863\n0.4912\n0.4961\n0.5010\n0.5059\n0.5107\n0.5156\n0.5205\n0.5254\n0.5303\n0.5352\n0.5400\n0.5449\n0.5498\n0.5547\n0.5596\n0.5645\n0.5693\n0.5742\n0.5791\n0.5840\n0.5889\n0.5938\n0.5986\n0.6035\n0.6084\n0.6133\n0.6182\n0.6230\

In [ ]:
os.chdir(destination_folder)

In [ ]:
int(min_x)

-885

In [ ]:
boun_loc

'-884.1100'

In [ ]:
my_file = open("sinir.SP1", "w")
line1 = """SWAN   1                                Swan standard spectral file, version
$   Data produced by SWAN version 41.20               
$   Project: project_002'A577;  run number: 1   
LOCATIONS                               locations in x-y-space
     1                                  number of locations
"""
line2="      " + boun_loc + "        0.0000"
line3="""
AFREQ                                   absolute frequencies in Hz
"""
line4 ="    " +str(size)+"                                  number of frequencies\n"
line5 = spec['Freq'].to_string(index=False).replace(" ", "    ")
line6 = """
QUANT
     3                                  number of quantities in table
VaDens                                  variance densities in m2/Hz
m2/Hz                                   unit
   -0.9900E+02                          exception value
CDIR                                    average Cartesian direction in degr
degr                                    unit
   -0.9990E+03                          exception value
DSPRDEGR                                directional spreading                   
degr                                    unit
   -0.9000E+01                          exception value
LOCATION     1
"""
line7 = spec['Freq'].apply(lambda x: '%.2E' % Decimal (x)).to_string(index=False).replace(" ", "").replace("\n",  " 0.00 22.5\n") + " 0.00 22.5"
wir = line1 + line2 + line3 + line4 + line5 + line6 + line7
my_file.write(wir)
print(wir)
my_file.close()

SWAN   1                                Swan standard spectral file, version
$   Data produced by SWAN version 41.20               
$   Project: project_002'A577;  run number: 1   
LOCATIONS                               locations in x-y-space
     1                                  number of locations
      -884.1100        0.0000
AFREQ                                   absolute frequencies in Hz
    201                                  number of frequencies
0.0176
0.0225
0.0273
0.0322
0.0371
0.0420
0.0469
0.0518
0.0566
0.0615
0.0664
0.0713
0.0762
0.0811
0.0859
0.0908
0.0957
0.1006
0.1055
0.1104
0.1152
0.1201
0.1250
0.1299
0.1348
0.1396
0.1445
0.1494
0.1543
0.1592
0.1641
0.1689
0.1738
0.1787
0.1836
0.1885
0.1934
0.1982
0.2031
0.2080
0.2129
0.2178
0.2227
0.2275
0.2324
0.2373
0.2422
0.2471
0.2520
0.2568
0.2617
0.2666
0.2715
0.2764
0.2813
0.2861
0.2910
0.2959
0.3008
0.3057
0.3105
0.3154
0.3203
0.3252
0.3301
0.3350
0.3398
0.3447
0.3496
0.3545
0.3594
0.3643
0.3691
0.3740
0.3789
0.3838
0.38

# Points.txt File

In [ ]:
sheetname

'09260100.ppp87'

In [ ]:
float(boun_loc)

-884.11

In [ ]:
my_file = open("points.txt", "w")
line1 = """-135.01 0.00
-145.42 0.00
-160.77 0.00
-169.5  0.00
-205.34 0.00
-220.23 0.00
-240.55 0.00
-264.7  0.00
-252.03 0.00
-320.37 0.00
-398.39 0.00
-190.2  0.00
-370.08 0.00
"""
line2 = """-480.34 0.00
"""
line3 = """-884.11 0.00
"""
if float(boun_loc) == -480.34:
  wir = line1+line2
elif float(boun_loc) == -884.11:
  wir = line1+line2+line3
else:
  wir = line1
print(wir)
my_file.write(wir)
my_file.close()

-135.01 0.00
-145.42 0.00
-160.77 0.00
-169.5  0.00
-205.34 0.00
-220.23 0.00
-240.55 0.00
-264.7  0.00
-252.03 0.00
-320.37 0.00
-398.39 0.00
-190.2  0.00
-370.08 0.00
-480.34 0.00
-884.11 0.00



In [ ]:
if float(boun_loc) < float(min_x):
  print("WARNING min_x value is incorrect")